In [0]:
!pip install transformers 

import time 
import datetime
import random

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np 
import pandas as pd 
import tensorflow as tf
import torch 

from sklearn.model_selection import train_test_split 

from torch.utils.data import TensorDataset, random_split 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup, AdamW

In [2]:
# check for GPU 
device_name = tf.test.gpu_device_name()

if device_name == "/device:GPU:0":
  print("Found GPU at {}".format(device_name))
else:
  raise SystemError("GPU device not found")
  
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("There are {} GPU(s) available".format(torch.cuda.device_count()))
  print("We will use the GPU", torch.cuda.get_device_name(0))
else: 
  device = torch.device("cpu")
  print("No GPU available, we use the CPU instead")

Found GPU at /device:GPU:0
There are 1 GPU(s) available
We will use the GPU Tesla K80


In [0]:
# loading the dataset 
url = "https://raw.githubusercontent.com/ant1code/tweet-sentiment/master/data/train.csv"
df = pd.read_csv(url)[:15000] # NOTE: try to increase this as much as possible

# converting sentiment (categorical) to label (numerical)
# 0: negative, 1: neutral, 2: positive 
# FIXED: mapping was arbitrary
sentiment_to_label = {"negative": 0, "neutral": 1, "positive": 2}
df["label"]  = df.sentiment.map(sentiment_to_label) 

# drop irrelevant columns and rows with missing values 
df.drop(["textID", "selected_text", "sentiment"], axis=1, inplace=True)
df = df[df.text.notna()]

# convert all text to lowercase because we use uncased DistilBert 
# NOTE: try cased DistilBert also 
df.text = df.text.str.lower()

# get length of longest sentence in total dataset (train+valid+test)
# all sentences in Bert models must have the same length after tokenization 
# NOTE: does this count as using the test set? 
max_length = max(df.text.apply(len))

# split into train and test sets (90-10 split)
# this already shuffles the dataset 
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label, test_size=0.1)

In [0]:
# load DistilBert tokenizer 
# NOTE: also try DistilBertTokenizerFast and distil-bert-cased
pretrained_weights = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights) 

In [0]:
# Bert formatting requirements: 
# - special tokens ([CLS], [SEP]) at start and end of every sentence 
# - all sentences have, or are padded to, the same length 
# - sentences have <= 512 tokens
# - each sentence has an "attention mask" that specifies where real tokens end 
#   and padding tokens begin 

token_ids = [] 
attention_masks = []

for text in x_train.values:
  encoded_dict = tokenizer.encode_plus(
      # NOTE: return_token_type_ids is None by default 
      # NOTE: what if the model is fed a sentence with #tokens > max_length?
      text, add_special_tokens=True, max_length=max_length,
      pad_to_max_length=True, return_attention_mask=True, return_tensors="pt"
  )

  token_ids.append(encoded_dict["input_ids"])
  attention_masks.append(encoded_dict["attention_mask"])

# create torch tensors size(train_set) x max_length
token_ids = torch.cat(token_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [0]:
labels = torch.tensor(y_train.values) 
dataset = TensorDataset(token_ids, attention_masks, labels)

# split into train and validation sets (90-10 split)
train_size = int(0.9 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

In [0]:
# create train and validation dataloaders
# DataLoaders are supposed to help reduce RAM usage by not loading the entire 
# dataset into memory 
# NOTE: also try batch_size=16
batch_size = 32 

# NOTE: is sampling randomly necessary? we already shuffled the dataset at the start 
train_dataloader = DataLoader(
    train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size
)

valid_dataloader = DataLoader(
    valid_dataset, sampler=SequentialSampler(valid_dataset), batch_size=batch_size
)

In [0]:
# load the (pretrained) classification model 
# DistilBertForSequenceClassification = DistilBert with a single linear layer 
# on top for sequence classification 
# NOTE: try BertForSequenceClassification? though it's a lot larger

model = DistilBertForSequenceClassification.from_pretrained(
    pretrained_weights, num_labels=3, output_attentions=False, output_hidden_states=False
)

model.cuda()

In [0]:
# optimizer with fixed weight decay - used in fine-tuning models
# NOTE: plot graphs to find best values for learning rate and epsilon?

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [0]:
# NOTE: plot graphs to find best number of epochs
# Bert authors recommend only 2-4 epochs for reasons unknown
epochs = 4
total_steps = len(train_dataloader) * epochs

# this scheduler has a learning rate that first increases linearly (this is 
# called the "warmup period") then decreases linearly 
# NOTE: try other schedules e.g. exponential decay
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

In [0]:
def flat_accuracy(logits, labels):
  # let prediction be the logit with highest value 
  pred_flat = np.argmax(logits, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
# for reproducibility purposes
# comment out for real training 

seed_val = 10 
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [13]:
# main training loop 
training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs): 
  print("")
  print("============== Epoch {:} / {:} ==============".format(epoch_i+1, epochs))
  print("Training...")

  t0 = time.time()
  total_train_loss = 0

  # put model in training mode 
  model.train() 
  
  for step, batch in enumerate(train_dataloader):
    # unpack input and labels 
    b_input_ids  = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels     = batch[2].to(device)

    # clear previously calculated gradients - must be done before backward pass! 
    model.zero_grad()

    # forward pass to evaluate model on this batch 
    # NOTE: without conversion to long, error occurs. but is there a more elegant method? 
    loss, logits = model(
        input_ids      = b_input_ids, 
        attention_mask = b_input_mask, 
        labels         = b_labels.long() 
    ) 

    # increment by this batch's training loss 
    total_train_loss += loss.item() 

    # backward pass to calculate gradients 
    loss.backward() 

    # limit gradients to 1.0 to avoid infinite gradients 
    # NOTE: try commenting this out too lol
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters (according to optimizer) 
    optimizer.step() 
    
    # update learning rate (according to scheduler)
    scheduler.step() 

    # print training progress every 40 batches 
    if step % 40 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print("  batch {:>5,} of {:>5,};    elapsed: {:}".format(step, len(train_dataloader), elapsed))

  # average training loss for this epoch 
  avg_train_loss = total_train_loss / len(train_dataloader)
  
  # time taken for training in this epoch
  training_time = format_time(time.time() - t0)

  print("")
  print("  average training loss: {0:.2f}".format(avg_train_loss))
  print("  time taken for training: {:}".format(training_time))

  print("")
  print("Running validation...")

  t0 = time.time() 
  total_eval_accuracy = 0
  total_eval_loss = 0 
  nb_eval_steps = 0 

  # put model in evaluation mode
  model.eval()

  for batch in valid_dataloader: 
    # unpack input and labels 
    b_input_ids  = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels     = batch[2].to(device)

    # disable gradient calculation, since there are no backward passes 
    with torch.no_grad(): 
      # forward pass to evaluate model on this batch 
      loss, logits = model(
          b_input_ids, attention_mask=b_input_mask, labels=b_labels.long()
      ) 
    
    # increment by this batch's validation loss 
    total_eval_loss += loss.item()

    # move logits and labels to cpu 
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to("cpu").numpy()

    total_eval_accuracy += flat_accuracy(logits, label_ids)

  # average validation statistics for this epoch
  avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
  avg_val_loss = total_eval_loss / len(valid_dataloader)
  valid_time = format_time(time.time() - t0)
  print("  validation accuracy: {0:.2f}".format(avg_val_accuracy))
  print("  validation loss: {0:.2f}".format(avg_val_loss))
  print("  time taken for validation: {:}".format(valid_time))
  
  valid_time = format_time(time.time() - t0)

  # record this epoch's train/validation statistics
  training_stats.append(
      {
          "epoch": epoch_i+1, 
          "training loss": avg_train_loss, 
          "valid. loss": avg_val_loss, 
          "valid. accuracy": avg_val_accuracy, 
          "training time": training_time, 
          "valid. time": valid_time
      }
  )

print("")
print("Training is complete.")
print("Training time: {:}".format(format_time(time.time() - total_t0)))


======= Epoch 1 / 4 =======
Training...
  Batch    40 of   380.    Elapsed: 0:00:34
  Batch    80 of   380.    Elapsed: 0:01:07
  Batch   120 of   380.    Elapsed: 0:01:40
  Batch   160 of   380.    Elapsed: 0:02:13
  Batch   200 of   380.    Elapsed: 0:02:46
  Batch   240 of   380.    Elapsed: 0:03:19
  Batch   280 of   380.    Elapsed: 0:03:52
  Batch   320 of   380.    Elapsed: 0:04:25
  Batch   360 of   380.    Elapsed: 0:04:58

  Average training loss: 0.66
  Training epoch took: 0:05:13

Running validation...
  Accuracy: 0.76
  Validation loss: 0.58

======= Epoch 2 / 4 =======
Training...
  Batch    40 of   380.    Elapsed: 0:00:34
  Batch    80 of   380.    Elapsed: 0:01:07
  Batch   120 of   380.    Elapsed: 0:01:39
  Batch   160 of   380.    Elapsed: 0:02:12
  Batch   200 of   380.    Elapsed: 0:02:45
  Batch   240 of   380.    Elapsed: 0:03:18
  Batch   280 of   380.    Elapsed: 0:03:51
  Batch   320 of   380.    Elapsed: 0:04:24
  Batch   360 of   380.    Elapsed: 0:04:57


In [14]:
# summary of fine-tuning 
pd.set_option("precision", 2)
training_stats = pd.DataFrame(data=training_stats)
training_stats = training_stats.set_index("epoch")

training_stats

,training loss,valid. loss,valid. accuracy,training time,valid. time
epoch,,,,,
1,0.66,0.58,0.76,0:05:13,0:00:13
2,0.47,0.60,0.76,0:05:12,0:00:13
3,0.37,0.63,0.77,0:05:12,0:00:13
4,0.30,0.66,0.77,0:05:12,0:00:13


In [0]:
# plot training and validation loss to detect over/underfitting
% matplotlib inline

plt.plot(training_stats)